In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [2]:
data_path = "datas/fra-eng/fra.txt"

batch_size = 8
epochs = 10
latent_dim = 256
num_samples = 10000



In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t>0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1.0
    decoder_input_data[i, t+1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0



Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [4]:
encoder_inputs = keras.layers.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.layers.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _, = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.summary()



Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 92)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  357376      input_2[0][0]                    
                                                                 lstm[0][1]            

In [5]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

model.save(filepath="models/lstm_seq2seq.h5", include_optimizer=False, save_format="h5")


Epoch 1/10
1000/1000 [==============================] - 71s 71ms/step - loss: 0.7207 - accuracy: 0.8041 - val_loss: 0.6212 - val_accuracy: 0.8184

In [6]:
model = keras.models.load_model("models/lstm_seq2seq.h5")

encoder_inputs = model.input[0]

encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output

encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.layers.Input(shape=(latent_dim, ), name="input_3")
decoder_state_input_c = keras.layers.Input(shape=(latent_dim, ), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)

decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = keras.Model(
    [decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq]+states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        samped_char = reverse_target_char_index[sampled_token_index]

        decoded_sentence += samped_char

        if samped_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [7]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    print("Target sentence:", target_texts[seq_index])



-
Input sentence: Go.
Decoded sentence: Cours !

Target sentence: 	Va !

-
Input sentence: Go.
Decoded sentence: Cours !

Target sentence: 	Marche.

-
Input sentence: Go.
Decoded sentence: Cours !

Target sentence: 	Bouge !

-
Input sentence: Hi.
Decoded sentence: Attendez !

Target sentence: 	Salut !

-
Input sentence: Hi.
Decoded sentence: Attendez !

Target sentence: 	Salut.

-
Input sentence: Run!
Decoded sentence: Cours !

Target sentence: 	Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

Target sentence: 	Courez !

-
Input sentence: Run!
Decoded sentence: Cours !

Target sentence: 	Prenez vos jambes à vos cous !

-
Input sentence: Run!
Decoded sentence: Cours !

Target sentence: 	File !

-
Input sentence: Run!
Decoded sentence: Cours !

Target sentence: 	Filez !

-
Input sentence: Run!
Decoded sentence: Cours !

Target sentence: 	Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

Target sentence: 	Fuyez !

-
Input sentence: Run!
Decoded sentence: Cours !

Target